## 各种import

In [5]:
!pip install 'keras<2'

    100% |████████████████████████████████| 184kB 2.7MB/s 
    100% |████████████████████████████████| 2.8MB 395kB/s 
  Running setup.py bdist_wheel for keras ... - done
  Stored in directory: /content/.cache/pip/wheels/f6/c5/63/97d96b41bf822858027c70b04448c19deaccf1cf518148fa82
  Running setup.py bdist_wheel for theano ... - \ | / - \ | done
  Stored in directory: /content/.cache/pip/wheels/46/a2/7d/b4cac381d5151daa9f9e0b3e4e4b65edaea6355ae296c97cf2
Successfully built keras theano


In [6]:
from __future__ import division,print_function
import math, os, json, sys, re
import cPickle as pickle
from glob import glob
import numpy as np
from matplotlib import pyplot as plt
from operator import itemgetter, attrgetter, methodcaller
from collections import OrderedDict
import itertools
from itertools import chain

import pandas as pd
import PIL
from PIL import Image
from numpy.random import random, permutation, randn, normal, uniform, choice
from numpy import newaxis
import scipy
from scipy import misc, ndimage
from scipy.ndimage.interpolation import zoom
from scipy.ndimage import imread
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import OneHotEncoder
from sklearn.manifold import TSNE

from IPython.lib.display import FileLink

import theano
from theano import shared, tensor as T
from theano.tensor.nnet import conv2d, nnet
from theano.tensor.signal import pool

import keras
from keras import backend as K
from keras.utils.data_utils import get_file
from keras.utils import np_utils
from keras.utils.np_utils import to_categorical
from keras.models import Sequential, Model
from keras.layers import Input, Embedding, Reshape, merge, LSTM, Bidirectional
from keras.layers import TimeDistributed, Activation, SimpleRNN, GRU
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.regularizers import l2, activity_l2, l1, activity_l1
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD, RMSprop, Adam
from keras.utils.layer_utils import layer_from_config
from keras.metrics import categorical_crossentropy, categorical_accuracy
from keras.layers.convolutional import *
from keras.preprocessing import image, sequence
from keras.preprocessing.text import Tokenizer

Using TensorFlow backend.


In [7]:
%matplotlib inline

import os, sys 
sys.path.append(os.path.abspath("../examples"))

import utils; reload(utils)
from utils import *

from __future__ import division, print_function

ImportError: ignored

## 从Keras导入数据

In [8]:
from keras.datasets import mnist
(trn_data, trn_res), (test_data, test_res) = mnist.load_data()
(trn_data.shape, trn_res.shape, test_data.shape, test_res.shape)

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

## 处理数据

### One-Hot-Encoding

In [9]:
def onehot(x): return np.array(OneHotEncoder().fit_transform(x.reshape(-1,1)).todense())
trn_label = onehot(trn_res)
test_label = onehot(test_res)

print(trn_res[:5])
print(trn_label[:5])

[5 0 4 1 9]
[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


### 对图像加入新的维度
即灰度, 维度大小为1; 彩色图片则维度大小为3, 代表RGB, 3个通道
对于tensorflow, 前两个维度代表图片大小, 第三个维度代表色彩通道, 所以np.expand_dims(trn_data, 3)
对于thenao, 第一个维度代表色彩通道, 后两个维度代表图片大小, 所以np.expand_dims(trn_data, 1)

In [10]:
trn_data = np.expand_dims(trn_data, 3)
test_data = np.expand_dims(test_data, 3)
trn_data.shape

(60000, 28, 28, 1)

### 对输入图像进行正则化

In [0]:
input_mean = trn_data.mean().astype(np.float32)
input_std = trn_data.std().astype(np.float32)

def norm_input(x):
    return (x-input_mean)/input_std

### 将输入数据转化为batches

In [0]:
batch_size = 64
gen = image.ImageDataGenerator()
batches = gen.flow(trn_data, trn_label, batch_size=batch_size)
test_batches = gen.flow(test_data, test_label, batch_size=batch_size)

## 1. 简单神经网络模型

In [0]:
simpleNN = Sequential()
simpleNN.add(Lambda(norm_input, input_shape=(28, 28, 1)))
simpleNN.add(Flatten())
simpleNN.add(Dense(512, activation='relu'))
simpleNN.add(Dense(10, activation='softmax'))
simpleNN.compile(optimizer=SGD(), loss='categorical_crossentropy', metrics=['accuracy'])

### 1.1 直接训练

In [14]:
#simpleNN.optimizer.lr = 0.1
simpleNN.fit(trn_data, trn_label, validation_data=(test_data, test_label), nb_epoch=1)

Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 9s - loss: 0.3306 - acc: 0.9059 - val_loss: 0.2073 - val_acc: 0.9385


In [15]:
simpleNN.optimizer.lr = 0.01
simpleNN.fit(trn_data, trn_label, validation_data=(test_data, test_label), nb_epoch=3)

Train on 60000 samples, validate on 10000 samples
Epoch 1/3
60000/60000 [==============================] - 8s - loss: 0.1824 - acc: 0.9478 - val_loss: 0.1572 - val_acc: 0.9546
Epoch 2/3
60000/60000 [==============================] - 8s - loss: 0.1395 - acc: 0.9610 - val_loss: 0.1314 - val_acc: 0.9613
Epoch 3/3
33856/60000 [===============>..............] - ETA: 3s - loss: 0.1193 - acc: 0.9668

60000/60000 [==============================] - 8s - loss: 0.1140 - acc: 0.9684 - val_loss: 0.1140 - val_acc: 0.9658


### 1.2 生成batch再训练

利用ImageDataGenerator(), 使用fit_generator而不是fit
注意fit_generator的文档中参数由于版本升级发生了变化

In [17]:
simpleNN.optimizer.lr = 0.1
simpleNN.fit_generator(batches, trn_data.shape[0], nb_epoch=1, validation_data=test_batches, nb_val_samples=test_data.shape[0])

Epoch 1/1
60000/60000 [==============================] - 13s - loss: 0.2080 - acc: 0.9389 - val_loss: 0.1421 - val_acc: 0.9528


In [18]:
simpleNN.optimizer.lr = 0.01
simpleNN.fit_generator(batches, trn_data.shape[0], nb_epoch=3, validation_data=test_batches, nb_val_samples=test_data.shape[0])

Epoch 1/3
60000/60000 [==============================] - 14s - loss: 0.0846 - acc: 0.9747 - val_loss: 0.0835 - val_acc: 0.9744
Epoch 2/3
60000/60000 [==============================] - 14s - loss: 0.0560 - acc: 0.9838 - val_loss: 0.0734 - val_acc: 0.9777
Epoch 3/3
22464/60000 [==========>...................] - ETA: 8s - loss: 0.0395 - acc: 0.9898

60000/60000 [==============================] - 14s - loss: 0.0389 - acc: 0.9895 - val_loss: 0.0682 - val_acc: 0.9788


## 2. 类似vgg16的卷积神经网络模型

注意比较各类activation之间的关系, 只有最后一个线性层使用softmax使得结果One-Hot-Coded, 其余层使用relu

In [0]:
def getSimpleCNN():
  simpleCNN = Sequential([
      Lambda(norm_input, input_shape=(28, 28, 1)),
      Convolution2D(32, 3, 3, activation="relu"),
      Convolution2D(32, 3, 3, activation="relu"),
      MaxPooling2D(),
      Convolution2D(64, 3, 3, activation="relu"),
      Convolution2D(64, 3, 3, activation="relu"),
      MaxPooling2D(),
      Flatten(),
      Dense(512, activation="relu"),
      Dense(10, activation="softmax")])
  simpleCNN.compile(SGD(), loss='categorical_crossentropy', metrics=['accuracy'])
  return simpleCNN

In [20]:
simpleCNN = getSimpleCNN()
simpleCNN.optimizer.lr=0.1
simpleCNN.fit_generator(batches, trn_data.shape[0], nb_epoch=3, validation_data=test_batches, nb_val_samples=test_data.shape[0])

Epoch 1/3
60000/60000 [==============================] - 19s - loss: 0.1742 - acc: 0.9454 - val_loss: 0.0399 - val_acc: 0.9863
Epoch 2/3
60000/60000 [==============================] - 18s - loss: 0.0408 - acc: 0.9874 - val_loss: 0.0279 - val_acc: 0.9913
Epoch 3/3
 2176/60000 [>.............................] - ETA: 16s - loss: 0.0437 - acc: 0.9903

60000/60000 [==============================] - 18s - loss: 0.0262 - acc: 0.9919 - val_loss: 0.0276 - val_acc: 0.9912


In [21]:
simpleCNN.optimizer.lr=0.01
simpleCNN.fit_generator(batches, trn_data.shape[0], nb_epoch=10, validation_data=test_batches, nb_val_samples=test_data.shape[0])

Epoch 1/10
60000/60000 [==============================] - 19s - loss: 0.0182 - acc: 0.9942 - val_loss: 0.0285 - val_acc: 0.9913
Epoch 2/10
60000/60000 [==============================] - 19s - loss: 0.0148 - acc: 0.9954 - val_loss: 0.0245 - val_acc: 0.9919
Epoch 3/10
  448/60000 [..............................] - ETA: 16s - loss: 0.0075 - acc: 0.9955

60000/60000 [==============================] - 19s - loss: 0.0098 - acc: 0.9969 - val_loss: 0.0252 - val_acc: 0.9924
Epoch 4/10
60000/60000 [==============================] - 19s - loss: 0.0088 - acc: 0.9970 - val_loss: 0.0254 - val_acc: 0.9929
Epoch 5/10
 4608/60000 [=>............................] - ETA: 16s - loss: 0.0044 - acc: 0.9983

60000/60000 [==============================] - 19s - loss: 0.0065 - acc: 0.9978 - val_loss: 0.0268 - val_acc: 0.9932
Epoch 6/10
60000/60000 [==============================] - 19s - loss: 0.0050 - acc: 0.9987 - val_loss: 0.0306 - val_acc: 0.9912
Epoch 7/10
 5696/60000 [=>............................] - ETA: 15s - loss: 0.0040 - acc: 0.9989

60000/60000 [==============================] - 19s - loss: 0.0037 - acc: 0.9988 - val_loss: 0.0239 - val_acc: 0.9935
Epoch 8/10
60000/60000 [==============================] - 19s - loss: 0.0035 - acc: 0.9990 - val_loss: 0.0306 - val_acc: 0.9927
Epoch 9/10
 5888/60000 [=>............................] - ETA: 15s - loss: 0.0019 - acc: 0.9995

60000/60000 [==============================] - 19s - loss: 0.0036 - acc: 0.9989 - val_loss: 0.0324 - val_acc: 0.9931
Epoch 10/10
60000/60000 [==============================] - 19s - loss: 0.0022 - acc: 0.9992 - val_loss: 0.0272 - val_acc: 0.9939


In [22]:
simpleCNN.optimizer.lr=0.001
simpleCNN.fit_generator(batches, trn_data.shape[0], nb_epoch=1, validation_data=test_batches, nb_val_samples=test_data.shape[0])

Epoch 1/1
60000/60000 [==============================] - 20s - loss: 0.0024 - acc: 0.9993 - val_loss: 0.0481 - val_acc: 0.9899


## Data Augmentation

同样是利用ImageDataGenerator, 传入Data Augmentation的参数. 参数规定了随机对图片进行旋转、缩放等操作等角度、幅度范围

In [0]:
batch_size = 64
gen = image.ImageDataGenerator(rotation_range=10, width_shift_range=0.1, height_shift_range=0.1, zoom_range=0.1, shear_range=0.5)
batches = gen.flow(trn_data, trn_label, batch_size=batch_size)
test_batches = gen.flow(test_data, test_label, batch_size=batch_size)

In [24]:
simpleCNN = getSimpleCNN()
simpleCNN.optimizer.lr=0.1
simpleCNN.fit_generator(batches, trn_data.shape[0], nb_epoch=3, validation_data=test_batches, nb_val_samples=test_data.shape[0])

Epoch 1/3
60000/60000 [==============================] - 20s - loss: 0.4193 - acc: 0.8631 - val_loss: 0.1189 - val_acc: 0.9632
Epoch 2/3
60000/60000 [==============================] - 20s - loss: 0.1137 - acc: 0.9641 - val_loss: 0.0990 - val_acc: 0.9686
Epoch 3/3
  256/60000 [..............................] - ETA: 17s - loss: 0.0933 - acc: 0.9727

60000/60000 [==============================] - 21s - loss: 0.0865 - acc: 0.9734 - val_loss: 0.0714 - val_acc: 0.9781


In [25]:
simpleCNN.optimizer.lr=0.01
simpleCNN.fit_generator(batches, trn_data.shape[0], nb_epoch=10, validation_data=test_batches, nb_val_samples=test_data.shape[0])

Epoch 1/10
60000/60000 [==============================] - 21s - loss: 0.0707 - acc: 0.9770 - val_loss: 0.0648 - val_acc: 0.9802
Epoch 2/10
59200/60000 [============================>.] - ETA: 0s - loss: 0.0594 - acc: 0.9816

60000/60000 [==============================] - 21s - loss: 0.0595 - acc: 0.9816 - val_loss: 0.0553 - val_acc: 0.9814
Epoch 3/10
60000/60000 [==============================] - 21s - loss: 0.0530 - acc: 0.9838 - val_loss: 0.0484 - val_acc: 0.9858
Epoch 4/10
17152/60000 [=======>......................] - ETA: 13s - loss: 0.0543 - acc: 0.9835

60000/60000 [==============================] - 21s - loss: 0.0532 - acc: 0.9833 - val_loss: 0.0469 - val_acc: 0.9844
Epoch 5/10
60000/60000 [==============================] - 21s - loss: 0.0445 - acc: 0.9858 - val_loss: 0.0411 - val_acc: 0.9865
Epoch 6/10
 7936/60000 [==>...........................] - ETA: 16s - loss: 0.0442 - acc: 0.9861

60000/60000 [==============================] - 21s - loss: 0.0445 - acc: 0.9859 - val_loss: 0.0357 - val_acc: 0.9886
Epoch 7/10
60000/60000 [==============================] - 21s - loss: 0.0425 - acc: 0.9873 - val_loss: 0.0387 - val_acc: 0.9872
Epoch 8/10
 4352/60000 [=>............................] - ETA: 18s - loss: 0.0376 - acc: 0.9892

60000/60000 [==============================] - 21s - loss: 0.0417 - acc: 0.9873 - val_loss: 0.0340 - val_acc: 0.9894
Epoch 9/10
60000/60000 [==============================] - 21s - loss: 0.0395 - acc: 0.9878 - val_loss: 0.0331 - val_acc: 0.9882
Epoch 10/10
 3776/60000 [>.............................] - ETA: 17s - loss: 0.0367 - acc: 0.9870

60000/60000 [==============================] - 21s - loss: 0.0360 - acc: 0.9888 - val_loss: 0.0339 - val_acc: 0.9896


In [26]:
simpleCNN.optimizer.lr=0.001
simpleCNN.fit_generator(batches, trn_data.shape[0], nb_epoch=1, validation_data=test_batches, nb_val_samples=test_data.shape[0])

Epoch 1/1
60000/60000 [==============================] - 22s - loss: 0.0368 - acc: 0.9882 - val_loss: 0.0368 - val_acc: 0.9878


## Batch Normalization

注意, 对Batch Normalization要设定其axis为色彩通道所对应的axis, 默认为-1也是可以的

In [0]:
bn_model = Sequential([
    Lambda(norm_input, input_shape=(28, 28, 1)),
    Convolution2D(32, 3, 3, activation='relu'),
    BatchNormalization(axis=1),
    Convolution2D(32, 3, 3, activation="relu"),
    MaxPooling2D(),
    BatchNormalization(),
    Convolution2D(64, 3, 3, activation='relu'),
    BatchNormalization(axis=1),
    Convolution2D(64, 3, 3, activation="relu"),
    MaxPooling2D(),
    BatchNormalization(axis=1),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(10, activation='softmax')
])
bn_model.compile(SGD(), loss='categorical_crossentropy', metrics=['accuracy'])

In [30]:
bn_model.optimizer.lr = 0.1
bn_model.fit_generator(batches, trn_label.shape[0], validation_data=test_batches, nb_val_samples=test_label.shape[0], nb_epoch=3)

Epoch 1/3
60000/60000 [==============================] - 31s - loss: 0.2138 - acc: 0.9341 - val_loss: 0.0839 - val_acc: 0.9738
Epoch 2/3
31936/60000 [==============>...............] - ETA: 12s - loss: 0.0856 - acc: 0.9732

60000/60000 [==============================] - 30s - loss: 0.0774 - acc: 0.9759 - val_loss: 0.0638 - val_acc: 0.9784
Epoch 3/3
49344/60000 [=======================>......] - ETA: 5s - loss: 0.0589 - acc: 0.9812

60000/60000 [==============================] - 30s - loss: 0.0585 - acc: 0.9814 - val_loss: 0.0455 - val_acc: 0.9847


In [31]:
bn_model.optimizer.lr = 0.01
bn_model.fit_generator(batches, trn_label.shape[0], validation_data=test_batches, nb_val_samples=test_label.shape[0], nb_epoch=10)

Epoch 1/10
60000/60000 [==============================] - 29s - loss: 0.0529 - acc: 0.9835 - val_loss: 0.0493 - val_acc: 0.9859
Epoch 2/10
32384/60000 [===============>..............] - ETA: 12s - loss: 0.0456 - acc: 0.9859

60000/60000 [==============================] - 29s - loss: 0.0450 - acc: 0.9859 - val_loss: 0.0475 - val_acc: 0.9851
Epoch 3/10
50752/60000 [========================>.....] - ETA: 4s - loss: 0.0419 - acc: 0.9866

60000/60000 [==============================] - 29s - loss: 0.0416 - acc: 0.9866 - val_loss: 0.0310 - val_acc: 0.9895
Epoch 4/10
56928/60000 [===========================>..] - ETA: 1s - loss: 0.0393 - acc: 0.9880

60000/60000 [==============================] - 30s - loss: 0.0389 - acc: 0.9882 - val_loss: 0.0327 - val_acc: 0.9899
Epoch 5/10
59424/60000 [============================>.] - ETA: 0s - loss: 0.0362 - acc: 0.9887

60000/60000 [==============================] - 29s - loss: 0.0364 - acc: 0.9887 - val_loss: 0.0324 - val_acc: 0.9900
Epoch 6/10
59936/60000 [============================>.] - ETA: 0s - loss: 0.0337 - acc: 0.9894

60000/60000 [==============================] - 30s - loss: 0.0337 - acc: 0.9894 - val_loss: 0.0364 - val_acc: 0.9880
Epoch 7/10
60000/60000 [==============================] - 29s - loss: 0.0326 - acc: 0.9897 - val_loss: 0.0303 - val_acc: 0.9895
Epoch 8/10
  704/60000 [..............................] - ETA: 28s - loss: 0.0418 - acc: 0.9858

60000/60000 [==============================] - 30s - loss: 0.0303 - acc: 0.9905 - val_loss: 0.0265 - val_acc: 0.9918
Epoch 9/10
37376/60000 [=================>............] - ETA: 10s - loss: 0.0302 - acc: 0.9905

60000/60000 [==============================] - 29s - loss: 0.0304 - acc: 0.9905 - val_loss: 0.0297 - val_acc: 0.9906
Epoch 10/10
51392/60000 [========================>.....] - ETA: 3s - loss: 0.0316 - acc: 0.9900

60000/60000 [==============================] - 29s - loss: 0.0311 - acc: 0.9901 - val_loss: 0.0262 - val_acc: 0.9910


In [32]:
bn_model.optimizer.lr = 0.001
bn_model.fit_generator(batches, trn_label.shape[0], validation_data=test_batches, nb_val_samples=test_label.shape[0], nb_epoch=1)

Epoch 1/1
60000/60000 [==============================] - 30s - loss: 0.0265 - acc: 0.9914 - val_loss: 0.0293 - val_acc: 0.9897


# Drop Out

In [0]:
bndo_model = Sequential([
    Lambda(norm_input, input_shape=(28, 28, 1)),
    Convolution2D(32, 3, 3, activation='relu'),
    BatchNormalization(axis=1),
    Convolution2D(32, 3, 3, activation="relu"),
    MaxPooling2D(),
    BatchNormalization(),
    Convolution2D(64, 3, 3, activation='relu'),
    BatchNormalization(axis=1),
    Convolution2D(64, 3, 3, activation="relu"),
    MaxPooling2D(),
    BatchNormalization(axis=1),
    Flatten(),
    Dropout(0.5),
    Dense(512, activation='relu'),
    Dense(10, activation='softmax')
])
bndo_model.compile(SGD(), loss='categorical_crossentropy', metrics=['accuracy'])

In [35]:
bndo_model.optimizer.lr=0.1
bndo_model.fit(trn_data, trn_label, validation_data=(test_data, test_label), nb_epoch=3)

Train on 60000 samples, validate on 10000 samples
Epoch 1/3
60000/60000 [==============================] - 35s - loss: 0.1177 - acc: 0.9640 - val_loss: 0.0311 - val_acc: 0.9888
Epoch 2/3
 3872/60000 [>.............................] - ETA: 31s - loss: 0.0492 - acc: 0.9850

60000/60000 [==============================] - 34s - loss: 0.0460 - acc: 0.9858 - val_loss: 0.0259 - val_acc: 0.9911
Epoch 3/3
23776/60000 [==========>...................] - ETA: 20s - loss: 0.0350 - acc: 0.9881

60000/60000 [==============================] - 34s - loss: 0.0353 - acc: 0.9885 - val_loss: 0.0231 - val_acc: 0.9928


In [36]:
bndo_model.optimizer.lr=0.01
bndo_model.fit(trn_data, trn_label, validation_data=(test_data, test_label), nb_epoch=10)

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 34s - loss: 0.0293 - acc: 0.9909 - val_loss: 0.0192 - val_acc: 0.9932
Epoch 2/10
 4224/60000 [=>............................] - ETA: 30s - loss: 0.0148 - acc: 0.9953

60000/60000 [==============================] - 34s - loss: 0.0232 - acc: 0.9926 - val_loss: 0.0172 - val_acc: 0.9943
Epoch 3/10
24832/60000 [===========>..................] - ETA: 19s - loss: 0.0216 - acc: 0.9936

60000/60000 [==============================] - 34s - loss: 0.0207 - acc: 0.9935 - val_loss: 0.0199 - val_acc: 0.9937
Epoch 4/10
31936/60000 [==============>...............] - ETA: 15s - loss: 0.0182 - acc: 0.9937

60000/60000 [==============================] - 35s - loss: 0.0192 - acc: 0.9936 - val_loss: 0.0191 - val_acc: 0.9950
Epoch 5/10
34656/60000 [================>.............] - ETA: 14s - loss: 0.0171 - acc: 0.9945

60000/60000 [==============================] - 35s - loss: 0.0171 - acc: 0.9946 - val_loss: 0.0145 - val_acc: 0.9952
Epoch 6/10
35936/60000 [================>.............] - ETA: 13s - loss: 0.0132 - acc: 0.9956

60000/60000 [==============================] - 34s - loss: 0.0152 - acc: 0.9950 - val_loss: 0.0176 - val_acc: 0.9939
Epoch 7/10
36864/60000 [=================>............] - ETA: 12s - loss: 0.0130 - acc: 0.9956

60000/60000 [==============================] - 34s - loss: 0.0140 - acc: 0.9953 - val_loss: 0.0188 - val_acc: 0.9944
Epoch 8/10
39392/60000 [==================>...........] - ETA: 11s - loss: 0.0132 - acc: 0.9956

60000/60000 [==============================] - 33s - loss: 0.0134 - acc: 0.9956 - val_loss: 0.0159 - val_acc: 0.9949
Epoch 9/10
39584/60000 [==================>...........] - ETA: 11s - loss: 0.0112 - acc: 0.9966

60000/60000 [==============================] - 33s - loss: 0.0112 - acc: 0.9966 - val_loss: 0.0170 - val_acc: 0.9947
Epoch 10/10
38976/60000 [==================>...........] - ETA: 11s - loss: 0.0115 - acc: 0.9960

60000/60000 [==============================] - 34s - loss: 0.0109 - acc: 0.9962 - val_loss: 0.0159 - val_acc: 0.9947


In [37]:
bndo_model.optimizer.lr=0.001
bndo_model.fit(trn_data, trn_label, validation_data=(test_data, test_label), nb_epoch=1)

Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 34s - loss: 0.0101 - acc: 0.9964 - val_loss: 0.0167 - val_acc: 0.9940
